# Initialize env

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")


# Simple prompt

In [8]:
from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model='claude-3-7-sonnet-20250219', temperature=0)

In [12]:
response = model.invoke("Write a poem about a cat")

In [13]:
response

AIMessage(content="# Whiskers in the Moonlight\n\nSilent paws upon the floor,\nAmber eyes that watch and wait,\nMidnight fur that's soft and sleek,\nA purring soul, my feline mate.\n\nThrough windows gazing at the birds,\nA hunter's heart beneath the calm,\nStretching long in sunlight's beams,\nYour presence is a soothing balm.\n\nMysterious in your quiet ways,\nIndependent, proud, and free,\nYet curled against me as I sleep,\nA tender bond 'tween you and me.\n\nPlayful one moment, regal the next,\nA contradiction wrapped in grace,\nIn this home we share together,\nYou've found your perfect resting place.", additional_kwargs={}, response_metadata={'id': 'msg_014brD7kXomQitXDM7QPcZ3p', 'model': 'claude-3-7-sonnet-20250219', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 15, 'output_tokens': 167}}, id='run-097b49bb-7a19-43e9-b614-cc46aa70e6f4-0', usage_metadata={'input_tokens': 15, 'output_tokens

In [17]:
from IPython.display import Markdown
Markdown(response.content)

# Whiskers in the Moonlight

Silent paws upon the floor,
Amber eyes that watch and wait,
Midnight fur that's soft and sleek,
A purring soul, my feline mate.

Through windows gazing at the birds,
A hunter's heart beneath the calm,
Stretching long in sunlight's beams,
Your presence is a soothing balm.

Mysterious in your quiet ways,
Independent, proud, and free,
Yet curled against me as I sleep,
A tender bond 'tween you and me.

Playful one moment, regal the next,
A contradiction wrapped in grace,
In this home we share together,
You've found your perfect resting place.

# Prompt Template

In [18]:
from langchain_core.prompts import PromptTemplate

template = """
    Tell me a joke about {topic}
"""

prompt_template = PromptTemplate.from_template(template)



Example prompt =  Write a detailed report on the {company_name} using the following information: {information}

In [21]:
prompt = prompt_template.invoke({"topic": "Tiger"})

prompt

StringPromptValue(text='\n    Tell me a joke about Tiger\n')

In [22]:
response_joke = model.invoke(prompt)

response_joke.content

"Why don't tigers like fast food?\n\nBecause they can't catch it!"

# ChatPromptTemplate

In [27]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """ 
    You are a helpful assiatnt that can translate a sentence from English to {language}.

"""

user_prompt = """
    Translate the following sentence to {language}: {sentence}
"""

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", user_prompt)
])

prompt = prompt_template.invoke({"sentence": "I love programming in Python", "language": "Spanish"})

prompt


ChatPromptValue(messages=[SystemMessage(content=' \n    You are a helpful assiatnt that can translate a sentence from English to Spanish.\n\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='\n    Translate the following sentence to Spanish: I love programming in Python\n', additional_kwargs={}, response_metadata={})])

In [28]:
prompt.messages

[SystemMessage(content=' \n    You are a helpful assiatnt that can translate a sentence from English to Spanish.\n\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='\n    Translate the following sentence to Spanish: I love programming in Python\n', additional_kwargs={}, response_metadata={})]

In [29]:
response = model.invoke(prompt)

response.content



'I love programming in Python → Me encanta programar en Python'

In [30]:
response

AIMessage(content='I love programming in Python → Me encanta programar en Python', additional_kwargs={}, response_metadata={'id': 'msg_01QR1ce753Spsxh5hppueoY7', 'model': 'claude-3-7-sonnet-20250219', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 40, 'output_tokens': 16}}, id='run-68b0dc9e-ec46-4e03-a689-ce45f20e4779-0', usage_metadata={'input_tokens': 40, 'output_tokens': 16, 'total_tokens': 56, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}})

# Message Formats

In [31]:
from langchain_core.messages import HumanMessage, SystemMessage

system_prompt = """ 
    You are a helpful assiatnt that can translate a sentence from English to {language}.

"""

user_prompt = """
    Translate the following sentence to {language}: {sentence}
"""

system_message = SystemMessage(content=system_prompt.format(language="French"))
user_message = HumanMessage(content=user_prompt.format(sentence="I love programming in Python", language="French"))


messages = [system_message, user_message]

response = model.invoke(messages)

response.content

# Chaining



'I\'ll translate that sentence to French for you:\n\n"J\'aime programmer en Python."'

# Scrape contents from a url and summarize it

In [35]:
from newspaper import Article

def scrape_text(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.text

system_prompt = """
    You are a helpful assistant that can summarize a text.

    Guidelines:
    - Summarize the text in a concise manner.
    - Keep it short and to the point.

    Output Format:
     #Introduction
     #Body
      - Bullet points
     #Key Takeaways
     #Conclusion

"""

user_prompt = """
    Summarize the following text: {text}
"""

system_message = SystemMessage(content=system_prompt)

url = "https://www.anthropic.com/news/claude-3-7-sonnet"

user_message = HumanMessage(content=user_prompt.format(text=scrape_text(url)))

messages = [system_message, user_message]

response = model.invoke(messages)

response.content

'# Introduction\n- Anthropic announces Claude 3.7 Sonnet, their most intelligent model and first hybrid reasoning model\n- Introduces Claude Code, a command line tool for agentic coding in limited research preview\n\n# Body\n- Key features of Claude 3.7 Sonnet:\n  - Available in standard and extended thinking modes\n  - Shows significant improvements in coding and web development\n  - Maintains same pricing as predecessors ($3/million input tokens, $15/million output tokens)\n  - Available across all Claude plans and partner platforms\n  - API users can control how long the model thinks before responding\n  - Achieves state-of-the-art performance on SWE-bench Verified and TAU-bench\n\n- Claude Code capabilities:\n  - Can search/read code, edit files, write/run tests, and use command line tools\n  - Reduces development time for complex tasks\n  - Currently in limited research preview\n\n# Key Takeaways\n- Claude 3.7 Sonnet integrates quick responses and deep reasoning in a single model\

In [36]:
from IPython.display import Markdown
Markdown(response.content)

# Introduction
- Anthropic announces Claude 3.7 Sonnet, their most intelligent model and first hybrid reasoning model
- Introduces Claude Code, a command line tool for agentic coding in limited research preview

# Body
- Key features of Claude 3.7 Sonnet:
  - Available in standard and extended thinking modes
  - Shows significant improvements in coding and web development
  - Maintains same pricing as predecessors ($3/million input tokens, $15/million output tokens)
  - Available across all Claude plans and partner platforms
  - API users can control how long the model thinks before responding
  - Achieves state-of-the-art performance on SWE-bench Verified and TAU-bench

- Claude Code capabilities:
  - Can search/read code, edit files, write/run tests, and use command line tools
  - Reduces development time for complex tasks
  - Currently in limited research preview

# Key Takeaways
- Claude 3.7 Sonnet integrates quick responses and deep reasoning in a single model
- Significant improvements in real-world coding tasks confirmed by partners
- GitHub integration now available on all Claude plans
- Safety improvements include 45% reduction in unnecessary refusals

# Conclusion
- These releases represent progress toward AI systems that can augment human capabilities
- Anthropic continues to focus on responsible development with detailed system card documentation